Data manipulation to consolidate the file

In [1]:
import modin.pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import ray

ray.init()
    
dfe = pd.read_csv('./oldData/3cixty_cotedazur_events.csv', dtype=str)

dfe = dfe.groupby('event', as_index=False).agg({
    'label': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'description': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'category': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'subject': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'placeLabel': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'placeLocality': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'timeBegin': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'timeEnd': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else ''
})

#rename the column event to url
dfe['url'] = dfe['event']
dfe.pop('event')
dfe['label'] = dfe['label'].apply(lambda x: 'Event name: ' + str(x) if pd.notna(x) else x)
dfe['description'] = dfe['description'].apply(lambda x: ' Event description: ' + str(x) if pd.notna(x) else x)
dfe['description'] = dfe['description'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text() if pd.notna(x) else x)
dfe['category'] = dfe['category'].apply(lambda x: 'Event category: ' + str(x) if pd.notna(x) else x)
dfe['subject'] = dfe['subject'].apply(lambda x: 'Event subject: ' + str(x) if pd.notna(x) else x)
dfe['placeLabel'] = dfe['placeLabel'].apply(lambda x: 'Event place: ' + str(x) if pd.notna(x) else x)
dfe['placeLocality'] = dfe['placeLocality'].apply(lambda x: 'Event place location: ' + str(x) if pd.notna(x) else x)
dfe['timeBegin'] = dfe['timeBegin'].apply(lambda x: 'Event time begin: ' + str(x) if pd.notna(x) else x)
dfe['timeEnd'] = dfe['timeEnd'].apply(lambda x: 'Event time end: ' + str(x) if pd.notna(x) else x)

print(dfe.head())

# Save the result to a new CSV file
dfe.to_csv('./data/events.csv', index=False)

dfp = pd.read_csv('./oldData/3cixty_cotedazur_places.csv', dtype=str)

dfp = dfp.groupby('place', as_index=False, sort=False).agg({
    'label': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'description': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'businessTypeLabel': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'locality': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else ''
})

dfp['url'] = dfp['place']
dfp.pop('place')
dfp['label'] = dfp['label'].apply(lambda x: 'Place name: ' + str(x) if pd.notna(x) else x)
dfp['description'] = dfp['description'].apply(lambda x: 'Place description: ' + str(x) if pd.notna(x) else x)
dfp['businessTypeLabel'] = dfp['businessTypeLabel'].apply(lambda x: 'Place business type: ' + str(x) if pd.notna(x) else x)
dfp['locality'] = dfp['locality'].apply(lambda x: 'Place location: ' + str(x) if pd.notna(x) else x)

print(dfp.head())

dfp.to_csv('./data/places.csv', index=False)


2024-01-13 22:16:16,586	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


                                               label  \
0  Event name: Séjour : le Brame du Cerf à la Rés...   
1                    Event name: MEDIATHEQUE D'AURON   
2  Event name: ALL FRENCH RIVIERA FOR ONE DAY SIG...   
3  Event name: Conférence "Demain l'Humain" - Dem...   
4  Event name: Visite privée : excursion d'une de...   

                                         description  \
0   Event description: Séjour VIP  Tout le cérém...   
1                                Event description:    
2   Event description: Your guide will meet you a...   
3   Event description: Lieu: Palais des Congrès -...   
4   Event description: Découvrez la richesse des ...   

                                            category  \
0                  Event category: Nature et détente   
1  Event category: Activités sportives, culturell...   
2                    Event category: Circuits privés   
3                                   Event category:    
4                    Event category: Circuits 

In [2]:
import glob
from llama_index import VectorStoreIndex, ServiceContext, Document, StorageContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
import chromadb
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
import pandas as pd
import numpy as np
from llama_index.node_parser import SentenceSplitter, NodeParser
from llama_index.indices.prompt_helper import PromptHelper
from langchain_community.chat_models import ChatOpenAI
from langchain import OpenAI, ConversationChain, PromptTemplate, LLMChain
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain_core.retrievers import BaseRetriever
from langchain_core.vectorstores import VectorStoreRetriever
from summarizer.sbert import SBertSummarizer
from sentence_transformers import SentenceTransformer
from langchain.agents import initialize_agent
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory
from langchain_openai import OpenAIEmbeddings

In [3]:
system_prompt = (
    "You are a knowledgeable Tourism Assistant designed to provide visitors with "
    "information, recommendations, and tips for exploring and enjoying their destination. "
    "The assistant is familiar with a wide range of topics including historical sites, "
    "cultural events, local cuisine, accommodations, transportation options, and hidden gems. "
    "It offers up-to-date and personalized information to help tourists make the most of their trip."
)

service_context = ServiceContext.from_defaults(llm=ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.6), 
                                                prompt_helper = PromptHelper(),
                                                embed_model= OpenAIEmbeddings(), #LangchainEmbedding(HuggingFaceEmbeddings(model_name='dangvantuan/sentence-camembert-large',model_kwargs = {'device': 'cuda:0'})),
                                                node_parser=SentenceSplitter(),
                                                system_prompt=system_prompt,
                                                #chunk_size_limit=4096
                                                )


# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("tourism_events", embedding_function=service_context.embed_model) 
#embedding_functions.HuggingFaceEmbeddingFunction(model_name="dangvantuan/sentence-camembert-large")

# assign chroma as the vector_store to the context
storage_context = StorageContext.from_defaults(vector_store=ChromaVectorStore(chroma_collection=chroma_collection))

If we need to compute the index:

In [6]:
documents = []

# Get a list of all CSV files in the directory
for file in glob.glob('./data/*.csv'):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file, dtype=str, parse_dates=True)

    # Convert the DataFrame into a list of Document objects
    docs = [Document(doc_id=str(i), text=str(row.to_dict()), metadata={"url": row['url']}) for i, row in df.iterrows()]

    # Add the documents to the list
    documents.extend(docs)

#batch_size = 5461  # Maximum batch size
#for i in range(0, len(documents), batch_size):
#    batch = documents[i:i+batch_size]
    # Now add the batch to the index
index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context, show_progress=True).refresh(documents)
    #index = VectorStoreIndex.from_vector_store(ChromaVectorStore(chroma_collection=chroma_collection), storage_context=storage_context)
    #storage_context.persist(persist_dir=f"./chroma_db")
#storage_context.persist(persist_dir=f"./chroma_db")

Parsing nodes:   0%|          | 0/326089 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

if the index is already there:

In [10]:
index = VectorStoreIndex.from_vector_store(ChromaVectorStore(chroma_collection=chroma_collection), storage_context=storage_context)

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import OpenAI
from langchain.chains import ConversationChain

# Initialize the OpenAI model
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Initialize the memory with a max token limit
memory = ConversationSummaryBufferMemory(llm=llm, return_messages=True)

# Initialize the conversation chain with the LLM and memory
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Conversation loop with conversation history
conversation_memory = []
# Conversation loop
while True:
    user_input = input("Please enter your question: ")
    if user_input.lower() == "exit":
        break
    elif user_input.lower() == "reset":
        memory.clear()  # Clear the conversation history
        print("The conversation has been reset.")
        continue
    else:
        # Generate the response using the conversation chain
        response = index.as_query_engine(verbose=True, memory=memory).query(user_input)

        # Save the response to the memory
        memory.save_context({"input": user_input}, {"output": str(response)})

        # Print the response
        print(response)

    #print(index.as_query_engine(verbose=True, similarity_top_k=6, memory=memory).query(question))

print("Goodbye!")

c:\Users\chris\OneDrive\Desktop\EIT\2°Year\project\Chatbot\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Le sejour du brame de cerf is an event where visitors can experience the deer mating season in the forest. This is a unique opportunity to witness and listen to the impressive roars of the male deer during the mating ritual. It's a captivating natural spectacle that attracts nature enthusiasts and wildlife photographers.
The event lasts from 21:00 to 21:00.
Goodbye!
